In [4]:
import pandas as pd
from selenium import webdriver
import glob
import os
from tqdm import tqdm

geckodriver_path=r'C:\Users\Walid\Desktop\Selenium\geckodriver.exe'

# Data Prepration

This notebook is to show the first step of the process, by taking the initial dataframe and linking it with the folder pages and then make the verification that the annotation is working fine, in case some pages have attributes hidden they are dismissed.

In [5]:
df_annotated = pd.read_excel("df_annotated.xlsx")

urls = []
for name in glob.glob('pages/*.html'):
    urls.append(name)
urls.sort(key=lambda x:int(x.split('\\')[1].split('.')[0]))
df_annotated['URL_Local'] = urls

In [6]:
def extraction_verification(row_data_annotated, driver, display=True):
    """
    This function aims to verify that the annotation and web page downloading are good
    Takes in:
        row_data_annotated: one row from the annotated data (pandas row)
        driver: Selenium driver
        display: printing the results in consolse (boolean)
    """
    url_local = row_data_annotated['URL_Local']
    url_live = row_data_annotated['URL_Live']
    title_cssselector = row_data_annotated['Title']
    description_cssselector = row_data_annotated['Description']
    price_cssselector = row_data_annotated['Price']
    
    driver.get(os.getcwd()+'\\'+url_local)
    
    if not pd.isna(title_cssselector):
        title = driver.find_element_by_css_selector(title_cssselector).text
    else:
        title = "Not Found"
    if not pd.isna(description_cssselector):
        description = driver.find_element_by_css_selector(description_cssselector).text
    else:
        description = "Not Found"
    if not pd.isna(price_cssselector):
        price = driver.find_element_by_css_selector(price_cssselector).text
    else:
        price = "Not Found"
    
    if display:
        print('-------------------', url_live, '----------------------------')
        print('------------------', url_local, '----------------------------')
        print('Title: ', title)
        print('--------------------------------------------')
        print('Decription:', description)
        print('--------------------------------------------')
        print("Price :", price)
    
        
    return url_local, url_live, row_data_annotated['URL_Parent'], title, description, price

def extraction_verification_all(df_annotated, geckodriver_path=r'C:\Users\Walid\Desktop\Selenium\geckodriver.exe'):
    driver = webdriver.Firefox(executable_path=geckodriver_path)
    out = []
    indices_to_delete = []
    for i in tqdm(range(df_annotated.shape[0])):
        try:
            row_data_annotated = df_annotated.iloc[i]
            out.append(extraction_verification(row_data_annotated, driver, display=False))
        except:
            print('-------------------',i,'---------------')
            indices_to_delete.append(i)
    driver.close()
    df_verification = pd.DataFrame(out, columns=['URL_Local','URL_Live','URL_Parent','Title_Content','Description_Content','Price_Content'])
    bad_indices = df_annotated.index.isin(indices_to_delete)
    df_annotated_new = df_annotated[~bad_indices]
    df_annotated_new.reset_index(inplace=True, drop=True)
    df_annotated_new.to_excel('df_annotated_cleaned.xlsx', index=False)
    df_verification.to_excel('df_verification.xlsx', index=False)
    return out

In [9]:
#Verification for one page
driver = webdriver.Firefox(executable_path=geckodriver_path)
extraction_verification(df_annotated.iloc[0], driver)

In [11]:
extraction_verification_all(df_annotated)

 59%|████████████████████████████████████████████████▋                                 | 35/59 [00:16<00:07,  3.40it/s]

------------------- 33 ---------------


 71%|██████████████████████████████████████████████████████████▎                       | 42/59 [00:18<00:04,  3.75it/s]

------------------- 40 ---------------


100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [00:31<00:00,  1.89it/s]


[('pages\\1.html',
  'https://www.lamaisondubillard.com/8-pool-anglais/1014-billes-super-aramith-pro-cup-508-mm.html',
  'https://www.lamaisondubillard.com/',
  'Jeu de billes Aramith 50,8 Pro Cup : les boules de billard de compétition',
  "Le jeu de billes officiel des championnats du monde\nLe jeu de luxe anglais « Aramith PRO CUP » que nous avons sélectionné ici, est utilisé dans les compétitions de billard 8-pool du monde entier.\nLes plus grands noms du billard, tels que Roger Dumortier et Christophe Lambert, les utilisent quotidiennement.\nLe set de boules de billard de compétition « Aramith PRO CUP » a été conçu pour répondre à toutes vos attentes. La bille blanche est pointée par 6 points rouges incrustés dans la masse.\nIls permettent de perfectionner vos tirs. Effectivement, les marquages vous permettent de voir le sens de rotation et la vitesse exacte de la bille blanche lors d’un tir.\nAvec ces points de repère, prenez goût à voir votre jeu progresser rapidement lors de vos